### Step 2: Attribute Tiff Data to HydroBasins Level 12

Attributing information from Tiff files to HydroBASINS Level 12.  Methods are primarly from the attribution and file_management modules in utils and use Rasterio, Rasterstat, Geopandas packages.

Requirements:

#### Required Python Package(s): 
    * rasterstats
    * rasterio
    * pandas
    * json
    
#### Optional Python Package(s) :
    * timeit (this allows user to test processing time and calibrate runs accordingly)
    
#### Required File(s):
    * 'data/var/tif_var_file_info.json' from step 1c, note if this file is in a different directory user must specify appropriate directory.  This example assumes the file is in 'data/var' and that the notebook remains as downloaded.
    * HydroBASINS level 12 data.  In this example we use the pickled geodataframe ('data/basins_lvl12_gdf.pkl') and list of IDs 'data/basins_lvl12.txt' from step 1D

In [1]:
import geopandas as gpd
import pandas as pd
import rasterio
import json
from rasterstats import zonal_stats
from timeit import default_timer as timer

from utils import file_management as f_mng
from utils import attribution as attr

First we will import tif file processing information and HydroBASIN data that we stored in step 1C (see step1_data_management.ipynb).  

We then determine bounds for each HydroBASIN.  In some cases TIF files do not have global coverage.  Instead of running the entire (CPU intense) zonal statistics process on each TIF we run a quick box intersection algorithm to test which TIF files likely have relevant data for each basin and only run the rest of the attribution process on these files. 

Note there is a lot of data being loaded, the next cell may take over a minute to load.

In [2]:
#Import file processing information
with open("data/var/tif_vars_file_info.json", "r") as all_file_info:
    json_file_info = json.load(all_file_info)
    
#Read in list of HYBAS_IDs (see step1_data_management.ipynb).  Provides list of IDs to process.
hybas_id_list=f_mng.read_pkl_df('data/basins_lvl12.txt')

#Read in pickled level 12 HydroBASINS (see step1_data_management.ipynb)
gdf = f_mng.read_pkl_gdf()

This example is only running on a few basins as an example. To run on all basins remove [0:3] selection from hybas_id_list[0:3].  When running all basins this can take hours or days using one processor.  See the example of using multiprocessing in Python to run attribution of multiple tif files for all basins (XXXXXX.py).  Note a few additional packages are required.  In addition multiprocessing may require additional configurations for Windows users.


In [3]:
import warnings
warnings.filterwarnings(action='once')

all_stats = []

#run attribution on each basin, one basin at a time for a subset of basins indexed 0 to 3
for basin in hybas_id_list[0:3]:
    
    #Select row from geodataframe with basin information, including bounding box info
    basin_info_gdf = gdf.loc[gdf['HYBAS_ID']==int(basin)]
    pfaf_12 = basin_info_gdf.iloc[0]['PFAF_ID']
    sub_area = float((basin_info_gdf.iloc[0])['SUB_AREA'])
    
    bbox_gdf = basin_info_gdf.bounds

    #Initiate basin stat object, this will help manage stats as created for the basin
    basin_info = attr.Stats(basin, pfaf_12, sub_area)

    #Add bounding box of basin to object
    basin_info.bounds((bbox_gdf['minx']),(bbox_gdf['maxx']),(bbox_gdf['miny']), (bbox_gdf['maxy']))
    #basin_info.bounds((basin_info_gdf['minx']),(basin_info_gdf['maxx']),(basin_info_gdf['miny']), (basin_info_gdf['maxy']))
    
    # loop through each record (representing a file in the folder containing variable data) in json_file_info 
    for file in json_file_info:
            
        var_bounds = file['bounds']

        #evaluate if data in file spatialy overlaps spatial object using bounds, process if overlap/intersection occurs
        basin_info.evaluate_intersection(var_bounds)
        #If needed run zonal stats
        basin_info.run_zonal_stats(basin_info_gdf, file)
    
    #Add stats to a common list
    all_stats = basin_info.add_to_all_stats(all_stats)
    basin_info = None
    

First record in all_stats.  Note that we are capturing more data than we likely need for our end product.  These information can help us test results and help others track where data are coming from.

Each Record contains the following:
 * id: hybas_id from hydrobasins (level 12)
 * pfaf_12: pfaf value from HydroBASINS
 * suite of summaries: 
         * label_stat0:value0, label_stat1:value1... label_statx:valuex
         * src_file: tif files used in summary and how they evaluated in the bounds_eval method

In [4]:
#Show stats for basin id 1121976320
all_stats[0]

{'lc2015_bare_cov': {'lc2015_bare_cov_mean': 0.0,
  'lc2015_bare_cov_count': 12881,
  'lc2015_bare_cov_nodata': 0.0,
  'src_file': [{'file_name': 'E020N20_ProbaV_LC100_epoch2015_global_v2.0.1_bare-coverfraction-layer_EPSG-4326.tif',
    'bounds_eval': 1}]},
 'lc2015_grass_cov': {'lc2015_grass_cov_mean': 39.11241363248195,
  'lc2015_grass_cov_count': 12881,
  'lc2015_grass_cov_nodata': 0.0,
  'src_file': [{'file_name': 'E020N20_ProbaV_LC100_epoch2015_global_v2.0.1_grass-coverfraction-layer_EPSG-4326.tif',
    'bounds_eval': 1}]},
 'lc2015_shrub_cov': {'lc2015_shrub_cov_mean': 32.858240819812124,
  'lc2015_shrub_cov_count': 12881,
  'lc2015_shrub_cov_nodata': 0.0,
  'src_file': [{'file_name': 'E020N20_ProbaV_LC100_epoch2015_global_v2.0.1_shrub-coverfraction-layer_EPSG-4326.tif',
    'bounds_eval': 1}]},
 'lc2015_tree_cov': {'lc2015_tree_cov_mean': 28.023445384675103,
  'lc2015_tree_cov_count': 12881,
  'lc2015_tree_cov_nodata': 0.0,
  'src_file': [{'file_name': 'E020N20_ProbaV_LC100_epoc

In [5]:
#Export information to json file

#with open('ouput/tif_att_stats.json', 'w') as outfile:
#    json.dump(all_stats, outfile)

In [6]:
#Export information to csv file
outfile_name = 'output/tif_att_stats.csv'
attr.json_stats_to_csv(all_stats, outfile_name, pfaf_field_nm = 'pfaf_12')
